In [2]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [3]:
mlflow.set_experiment("Exp 7 - SVM HP Tuning")

2025/06/28 14:28:38 INFO mlflow.tracking.fluent: Experiment with name 'Exp 7 - SVM HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://ms-mlflow-bucket/432194903668034714', creation_time=1751101120689, experiment_id='432194903668034714', last_update_time=1751101120689, lifecycle_stage='active', name='Exp 7 - SVM HP Tuning', tags={}>

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [9]:
df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

(36243, 2)

In [10]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [11]:
def objective_svm(trial):
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly", "sigmoid"])
    C = trial.suggest_loguniform("C", 1e-3, 1e3)
    decision_shape = trial.suggest_categorical("decision_function_shape", ["ovr", "ovo"])

    if kernel in ["rbf", "poly", "sigmoid"]:
        gamma = trial.suggest_loguniform("gamma", 1e-5, 1e0)
    else:
        gamma = "scale"

    if kernel == "poly":
        degree = trial.suggest_int("degree", 2, 5)
    else:
        degree = 3

    if kernel in ["poly", "sigmoid"]:
        coef0 = trial.suggest_uniform("coef0", 0.0, 1.0)
    else:
        coef0 = 0.0

    shrinking = trial.suggest_categorical("shrinking", [True, False])
    tol = trial.suggest_loguniform("tol", 1e-4, 1e-2)

    model = SVC(
            kernel=kernel,
            C=C,
            gamma=gamma,
            degree=degree,
            coef0=coef0,
            shrinking=shrinking,
            tol=tol,
            cache_size=500,
            class_weight='balanced',
            decision_function_shape=decision_shape
            )
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))

In [12]:
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()

[I 2025-06-28 14:29:47,520] A new study created in memory with name: no-name-8171ad24-e93b-4e14-852f-7b922f42334d
C:\Users\miles\AppData\Local\Temp\ipykernel_23900\2401696014.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 1e3)
C:\Users\miles\AppData\Local\Temp\ipykernel_23900\2401696014.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform("tol", 1e-4, 1e-2)
[I 2025-06-28 14:32:25,072] Trial 0 finished with value: 0.8143617021276596 and parameters: {'kernel': 'linear', 'C': 2.8063450463898203, 'decision_function_shape': 'ovo', 'shrinking': True, 'tol': 0.007559863269852418}. Best is trial 0 with 

🏃 View run SVM_SMOTE_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/432194903668034714/runs/bc175886ada643ea935157002ac6a3b5
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/432194903668034714
